
합성곱 신경망(Convolutional Neural Network, CNN)은 주로 이미지 인식 및 패턴 인식에 사용되는 딥러닝 모델입니다. CNN은 이미지와 같은 다차원 데이터를 처리하는 데 특히 효과적입니다. 아래는 CNN의 주요 개념과 구성 요소에 대한 간단한 소개입니다.

합성곱 연상
1. 완전 연결층과 합성곱 층 사이의 근본적인 차이는 다음과 같다
  1-1. Dense - 전역 패턴을 학습
  1-2. 합성곱 - 지역 패턴을 학습
  1-3. 이미지일 경우, 작은 2D 윈도우로 입력해서 패턴을 찾음
  1-4. 앞의 예에서 이 윈도우는 모두 3x3 크기


In [ ]:
# 8-1 간단한 컨브넷 만들기

from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

# 8-2 모델의 summary() 메서드 출력
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 3, 3, 128)         7385

In [ ]:
# 코드 8-3 MNIST 이미지에서 컨브넷 훈련하기
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype("float32") / 255
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

#8-4 컨브넷 평가하기
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"테스트 정확도: {test_acc:.3f}")

11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/5
938/938 [==============================] - 60s 63ms/step - loss: 0.1570 - accuracy: 0.9504
Epoch 2/5
938/938 [==============================] - 56s 60ms/step - loss: 0.0445 - accuracy: 0.9863
Epoch 3/5
938/938 [==============================] - 56s 59ms/step - loss: 0.0301 - accuracy: 0.9906
Epoch 4/5
938/938 [==============================] - 55s 59ms/step - loss: 0.0227 - accuracy: 0.9929
Epoch 5/5
313/313 [==============================] - 3s 10ms/step - loss: 0.0248 - accuracy: 0.9921
테스트 정확도: 0.992


평가
2장의 완전 연결 네트워크는 n%의 텍스트 정확도를 얻은 반면,
기본적인 컨브넷은 99.2%의 테스트 정확도를 얻었음
-> 에러율이 상대적으로 60%나 줄었음

합성곱 신경망
합성곱 연산
  1. 학습된 패턴은 평행 이동 불변성을 가짐
  (입력신호의  특정위치나위치에의존하지않음)
  2. 완전 연결 네트워크는 새로운 위치에 나타난 것은 새로운 패턴으로 학습
 3. 적은 수의 훈련 샘플을 사용해서 일반화 능력을 가진 표현을 학습

컨브넷은 패턴의 공간적 계층구조를 학습할 수 있음
첫번째합성곱층 - 에지같은 작은 지역 패턴을 학습
두번째합성곱층 - 첫번째층의 특성으로 구성된 더 큰 패턴을 학습  이런방식을 사용하여 컨브넷은 매우 복잡하고
추상적인 시각적 개념을 효과적으로 학습할 수 있음
근본적으로 우리가 보는 세상은 공간적 계층 구조를 가지고 있기 때문임

In [ ]:
# 8-1 간단한 컨브넷 만들기

from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

입력과동일한높이와너비를가진출력특성맵을얻고싶다면패딩(padding)을  사용할수있음  ⚫패딩은입력특성맵의가장자리에적절한개수의행과열을추가  ⚫모든입력타일에합성곱윈도우의중앙을위치시킬수있음  ⚫하나의행을추가하고오른쪽,왼쪽에하나의열을추가(그림8-6)

합성곱 연산
합성곱 스트라이드 이해하기
⚫ 출력 크기에 영향을 미치는 다른 요소는 스트라이드
⚫ 필터를 적용하는 간격을 나타내는 매개변수
⚫ 입력 데이터의 너비와 높이에 따라 필터 이동 간격 결정 - strides=(2, 2)
⚫ 디폴트 - strides=1
⚫ 그림 8-7에서 5×5 크기의 입력(패딩 없음)에 strides=2 를 사용한 3×3 크기의
윈도우로 합성곱하여 추출한 패치

x = layers.Conv2D(filters=32, kernel_size=3, strides=2, activation='relu')

In [3]:
# 8-14. 컨브넷에 추가할 데이터 증식 단계 정의
data_augmentation = keras.Sequential(
  [
      layers.RandomFlip("horizontal"),
      layers.RandomRotation(0,1),
      layers.RandomZoom(0.2),
  ]
)

# 8-15. 랜덤하게 증식된 훈련 이미지 출력하기
plt.figure(figsize=(10, 10))
for images, _ in train_dataset.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

# 8-16 이미지 증식과 드롭아웃을 포함한 컨브넷 만들기
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

# 8-17. 규제를 추가한 컨브넷 훈련
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch_with_augmentation.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=validation_dataset,
    callbacks=callbacks
)

"""
 결과를 그래프로 나타내 보겠음(그림 8-11)
⚫ 데이터 증식과 드롭아웃 덕분에 과대적합이 훨씬 늦은 60~70번째 에포크
근처에서 시작 (원본 모델은 10번째 에포크에서 시작)
⚫ 검증 정확도는 80~85% 범위에서 유지
⚫ 이전 모델보다 훨씬 성능이 좋아졌음
"""

# 3-18 테스트 세트에서 모델 훈련, 데이터 증식 사용, 테스트 세트의 정확도 확인
test_model = keras.models.load_model(
    "convnet_from_scratch_with_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"테스트 정확도: {test_acc:.3f}")

NameError: name 'keras' is not defined